<a href="https://colab.research.google.com/github/TeachingTextMining/TextClassification/blob/main/02-SA-Transformers-Basic/02-TextClassification-with-Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clasificación de textos utilizando Transformers

La clasificación de textos consiste en, dado un texto, asignarle una entre varias categorías. Algunos ejemplos de esta tarea son:

- dado un tweet, categorizar su connotación como positiva, negativa o neutra.
- dado un post de Facebook, clasificarlo como portador de un lenguaje ofensivo o no.  

En la actividad exploraremos cómo utilizar soluciones *out of the box* para esta tarea incluidas en la librería [Transformers](https://huggingface.co/transformers/) y su aplicación para clasificar reviews de [IMDB](https://www.imdb.com/) sobre películas en las categorías \[$positive$, $negative$\]. 

Puede encontrar más información sobre este problema en [Kaggle](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) y en [Large Movie Review Datase](http://ai.stanford.edu/~amaas/data/sentiment/).

**Instrucciones:**

- siga las indicaciones y comentarios en cada apartado.


**Después de esta actividad nos habremos familiarizado con:**
- seleccionar e instanciar modelos pre-entrenados para realizar clasificación de textos.
- cómo instanciar un pipeline para la clasificación de textos utilizando la librería Transformers.
- utilizar este pipeline para clasificar nuevos textos.

**Requerimientos**
- python 3.6.12 - 3.8
- transformers==4.26.0
- tensorflow==2.11
- pandas==1.3.5
- plotly==5.5.0
- scikit-learn==1.0.0

<a name="sec:setup"></a>
### Instalación de librerías e importación de dependencias.

Para comenzar, es preciso instalar las dependencias, realizar los imports necesarios y definir algunas funciones auxiliares.

Ejecute las siguientes casillas prestando atención a las instrucciones adicionales en los comentarios.

In [2]:
# instalar librerías. Esta casilla es últil por ejemplo si se ejecuta el cuaderno en Google Colab
# Note que existen otras dependencias como tensorflow, etc. que en este caso se encontrarían ya instaladas
#%%capture
#!pip install transformers==4.26.0 tensorflow==2.11 pandas==1.3.5 plotly==5.5.0 scikit-learn==1.0.0

!pip install transformers
!pip install tensorflow
!pip install pandas
!pip install plotly
!pip install scikit-learn

print('Done!')

Done!


In [8]:
# reset environment
#%reset -f

# para cargar datos y realizar pre-procesamiento básico
import pandas as pd
from collections import Counter

# para evaluar los modelos 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.utils.multiclass import unique_labels

# para construir gráficas y realizar análisis exploratorio de los datos
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from tqdm import tqdm

# algoritmos de clasificación, tokenizadores, etc.
from transformers import TextClassificationPipeline, DistilBertTokenizer, TFDistilBertForSequenceClassification

from transformers.tokenization_utils import TruncationStrategy
print('Done!')

Done!


In [9]:
def get_score_tripadvisor(row):
    '''
    Funcion para reducir el abanico de puntuaciones a predecir.
    0: Negativa
    1: Neutral
    
    2: Positiva
    '''
    score = int(row['score'])
    if score < 3:
        return 0
    elif score < 4:
        return 1
    else:
        return 2



# función auxiliar que evalúa los resultados de una clasificación
def evaluate_model(y_true, y_pred, y_score=None, pos_label='positive'):

  print('==== Sumario de la clasificación ==== ')
  print(classification_report(y_true, y_pred))

  print('Accuracy -> {:.2%}\n'.format(accuracy_score(y_true, y_pred)))

  # graficar matriz de confusión
  display_labels = sorted(unique_labels(y_true, y_pred), reverse=True)
  cm = confusion_matrix(y_true, y_pred, labels=display_labels)

  z = cm[::-1]
  x = display_labels
  y =  x[::-1].copy()
  z_text = [[str(y) for y in x] for x in z]

  fig_cm = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

  fig_cm.update_layout(
      height=400, width=400,
      showlegend=True,
      margin={'t':150, 'l':0},
      title={'text' : 'Matriz de Confusión', 'x':0.5, 'y':0.95, 'xanchor': 'center'},
      xaxis = {'title_text':'Valor Real', 'tickangle':45, 'side':'top'},
      yaxis = {'title_text':'Valor Predicho', 'tickmode':'linear'},
  )
  fig_cm.show()


  # curva roc (definido para clasificación binaria)
  fig_roc = None
  if y_score is not None:
    fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label=pos_label)
    fig_roc = px.area(
        x=fpr, y=tpr,
        title = f'Curva ROC (AUC={auc(fpr, tpr):.4f})',
        labels=dict(x='Ratio Falsos Positivos', y='Ratio Verdaderos Positivos'),
        width=400, height=400
    )
    fig_roc.add_shape(type='line', line=dict(dash='dash'), x0=0, x1=1, y0=0, y1=1)

    fig_roc.update_yaxes(scaleanchor="x", scaleratio=1)
    fig_roc.update_xaxes(constrain='domain')
    
    fig_roc.show()

print('Done!')

Done!


### Carga de datos y análisis exploratorio

El primer paso consiste en obtener los datos relacionados con nuestra tarea dejándolos en el formato adecuado.  Existen diferentes opciones, entre estas:

- montar nuestra partición de Google Drive y leer un fichero desde esta.

- leer los datos desde un fichero en una carpeta local.

- leer los datos directamente de un URL.

En este caso, se encuentran en un fichero separado por comas con la siguiente estructura:

| Phrase | Sentiment| 
| ------ | ------ |
| This movie is really not all that bad...    | positive |


Ejecute la siguiente casilla para leer los datos.



In [10]:
# descomente las siguientes 3 líneas para leer datos desde Google Drive, asumiendo que se trata de un fichero llamado review.csv localizado dentro de una carpeta llamada 'Datos' en su Google Drive
#from google.colab import drive
#drive.mount('/content/drive')
#path = '/content/drive/MyDrive/Datos/ejemplo_review_train.csv'

# descomente la siguiente línea para leer los datos desde un archivo local, por ejemplo, asumiendo que se encuentra dentro de un directorio llamado sample_data
path = '/Users/usuario/entorno/master-ua/Minería de textos/TextClassification-main/01-SA-Pipeline/sample_data/ejercicio_tripadvisor.csv'

# descomente la siguiente línea para leer datos desde un URL
#path = 'https://github.com/TeachingTextMining/TextClassification/raw/main/01-SA-Pipeline/sample_data/ejemplo_review_train.csv'

#importar el archivo .py de la siguiente ruta C:\Users\usuario\entorno\master-ua\Minería de textos\TextClassification-main\00-Utils\tripadvisor.py
import sys
sys.path.append('/Users/usuario/entorno/master-ua/Minería de textos/TextClassification-main/00-Utils')
import tripadvisor

#carga los datos con la función load_data del archivo tripadvisor.py
df = tripadvisor.load_data_tripadvisor(path)

# mostrar los primeros registros
data = df
data.head()





,text,score,Opinion,PROS,CONS,Sentiment
274573_7795529,Good for the money Having read the reviews -...,4,Good for the money Having read the reviews -...,Location and Price,Beds,2
274573_6611183,Central location and good price Hotel Suisse...,3,Central location and good price Hotel Suisse...,NaN,NaN,1
274573_5823077,A great hotel in the middle of everything I...,3,A great hotel in the middle of everything I...,Location,Having to be buzzed in,1
274573_5326023,Great location but expensive and some restric...,3,Great location but expensive and some restric...,NaN,NaN,1
274573_5211722,Great comfort and value Comfortable and clea...,4,Great comfort and value Comfortable and clea...,NaN,NaN,2


Una vez leídos los datos, ejecute la siguiente casilla para construir una gráfica que muestra la distribución de clases en el corpus.


In [11]:
text_col = 'Opinion'  # columna del dataframe que contiene el texto (depende del formato de los datos)
class_col = 'Sentiment'  # columna del dataframe que contiene la clase (depende del formato de los datos)

# obtener algunas estadísticas sobre los datos
categories = sorted(data[class_col].unique(), reverse=False)
hist= Counter(data[class_col]) 
print(f'Total de instancias -> {data.shape[0]}')
print('Distribución de clases:')
for item in sorted(hist.items(), key=lambda x: x[0]): print(f'    {item[0]}: {round(item[1]/len(data[class_col]), 3)}')

print(f'Categorías -> {categories}')
print(f'Comentario de ejemplo -> {data[text_col][0]}')
print(f'Categoría del comentario -> {data[class_col][0]}')

#cambia los datos de la class_col y asigna el valor negativa en caso de que sea 0, neutral en caso de que sea 1 y positiva en caso de que sea 2
#data[class_col] = data[class_col].apply(lambda x: 'negativa' if x == 0 else ('neutral' if x == 1 else 'positiva'))

fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in categories]))
fig.show()

print('Done!')



Total de instancias -> 10508
Distribución de clases:
    0: 0.113
    1: 0.1
    2: 0.787
Categorías -> [0, 1, 2]
Comentario de ejemplo ->  Good for the money  Having read the reviews - well it is Rome and you get what you pay for  Nice place and good location - remember that it is at the TOP of the Spanish steps - so if you prefer not to make the walk at the end of the day - pay more and stay at another hotel at the BOTTOM of the steps The staff were great and the bathroom was nice  The room was fine for what we paid Not recommended if you are looking for a  romantic  getaway  but for my sister and I - it was fine for all the tourist things we did  
Categoría del comentario -> 2


Done!


In [13]:
#iguala el numero de instancias a la clase que menos instancias tiene
data = data.groupby(class_col).apply(lambda x: x.sample(data[class_col].value_counts().min(), random_state=42)).reset_index(drop=True)

# obtener algunas estadísticas sobre los datos
categories = sorted(data[class_col].unique(), reverse=False)
hist= Counter(data[class_col]) 
print(f'Total de instancias -> {data.shape[0]}')
print('Distribución de clases:')
for item in sorted(hist.items(), key=lambda x: x[0]): print(f'    {item[0]}: {round(item[1]/len(data[class_col]), 3)}')

print(f'Categorías -> {categories}')
print(f'Comentario de ejemplo -> {data[text_col][0]}')
print(f'Categoría del comentario -> {data[class_col][0]}')

fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in categories]))
fig.show()


Total de instancias -> 3156
Distribución de clases:
    0: 0.333
    1: 0.333
    2: 0.333
Categorías -> [0, 1, 2]
Comentario de ejemplo ->  Mixed Review for the Exedra  Just like my rating says  this hotel is okay  but has some problems   Like the other reviewers  I felt the Exedra had a great location and was quite beautiful  The lobby looked like it had just been decorated  with beautiful new cream colored couches and furnishings  But please be prepared for the GIANT tacky headboard in your room - crocodile embossed leather  a bit much for my taste  The rooms were very nice  extremely spacious for Italy and not too pricey for a 5 star hotel - EUR 230 including breakfast  The breakfast was good and offered a sophisticated spread  smoked swordfish which was pretty good   I had previously stayed at the St Regis down the street  which is more expensive but has smaller rooms - however  the St Regis is more elegant  less modern than the Exedra  During my stay at the Exedra  the internet c

Finalmente, ejecute la siguiente casilla para crear los conjuntos de entrenamiento y validación que se utilizarán para entrenar y validar los modelos.

In [165]:
# obtener conjuntos de entrenamiento (90%) y validación (10%)
seed = 0  # fijar random_state para reproducibilidad
train, val = train_test_split(data, test_size=.1, stratify=data[class_col], random_state=seed)

print('Done!')

Done!


### Carga del modelo pre-entrenado
La librería Transformers provee diferentes modelos listos para usar en la tarea de clasificación de textos. Una forma flexible de lograrlo consiste en:

- seleccionar un modelo pre-entrenado adecuado para la tarea. Podemos examinar los modelos disponibles en [https://huggingface.co/models](https://huggingface.co/models). Estaremos utilizando el llamado [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) que permite clasificar un texto en idioma inglés de acuerdo con su connotación **positiva** o **negativa**.

- instanciar el modelo y su correspondiente tokenizador.

- crear un pipeline para la clasificación de textos, en este caso utilizando la clase [TextClassificationPipeline](https://huggingface.co/transformers/main_classes/pipelines.html#transformers.TextClassificationPipeline).

- utilizar el pipeline para clasificar textos.


Ejecute la siguiente celda para instanciar el modelo y el correspondiente tokenizador.

**Note que:**
- la práctica recomendada al crear un nuevo modelo para Transformers es hacerlo disponible mediante un fichero que contiene los elementos necesarios para su posterior uso, como son el modelo, el tokenizador y una tarjeta con metadatos sobre el modelo. 

- es conveniente indagar sobre el modelo base utilizado, en este caso **DistilBert**, esto permitirá seleccionar las clases adecuadas para instanciar el modelo.


In [166]:
# configuraciones
cfg = {} # diccionario para organizar los objetos que son parámetros del modelo, etc.
cfg['framework'] = 'tf'
cfg['task'] = 'sentiment-analysis'
cfg['trained_model_name'] = 'distilbert-base-uncased-finetuned-sst-2-english'
cfg['max_length'] = 512  # máxima longitud de secuencia recomendada por DistilBERT
cfg['truncation'] = TruncationStrategy.ONLY_FIRST

# cargar el tokenizador, disponible en Transformers. Establecer model_max_length para cuando el tokenizador sea llamado, trunque automáticamente.
cfg['tokenizer'] = DistilBertTokenizer.from_pretrained(cfg['trained_model_name'] , model_max_length=cfg['max_length'])

# cargar el modelo, disponible en Transformers
cfg['transformer'] = TFDistilBertForSequenceClassification.from_pretrained(cfg['trained_model_name'])

# instanciar el pipeline para la clasificación de textos
model = TextClassificationPipeline(model=cfg['transformer'], tokenizer=cfg['tokenizer'], framework=cfg['framework'], task=cfg['task'], top_k=None)

print('Done!')

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_179']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Done!


Ejecute la siguiente celda para clasificar una la frase. Alternativamente, puede modificar el texto incluyendo uno de su preferencia. Recuerde que debe ser en idioma inglés.

In [174]:
# ejemplo de texto a clasificar, # lista [texto 1, text 2, ..., texto n]
text = ['Brian De Palma\'s undeniable virtuosity can\'t really camouflage the fact that his plot here is a thinly disguised\
        \"Psycho\" carbon copy, but he does provide a genuinely terrifying climax. His "Blow Out", made the next year, was an improvement.']

m_pred = predict_model(model, cfg, text, pref='m' )

# el nombre de los campos dependerá de pref al llamar a predic_model y las clases. Ver comentarios en la definición de la función
pred_labels = m_pred['labels_m'].values[0]
pred_proba = m_pred['scores_m_positive'].values[0]

print(f'\nLa categoría de la frase es -> {pred_labels}')
print(f'El score asignado a la clase positiva es -> {pred_proba:.2f}')

print('Done!')

100%|██████████| 1/1 [00:00<00:00,  8.40it/s]

['Brian De Palma\'s undeniable virtuosity can\'t really camouflage the fact that his plot here is a thinly disguised        "Psycho" carbon copy, but he does provide a genuinely terrifying climax. His "Blow Out", made the next year, was an improvement.']

La categoría de la frase es -> positive
El score asignado a la clase positiva es -> 0.99
Done!


In [185]:
import numpy as np
#define la función predict_model para tres clases
#por ejemplo, para tres clases [{'label':'NEUTRAL', 'score':0.5}, {'label':'NEGATIVE', 'score':0.1}, {'label':'POSITIVE', 'score':0.4}]
#los parametros seran (model, cfg, data, batch_size=128, pref='m')
def predict_model(model, cfg, data, batch_size=128, pref='m'):
    #creamos una variable para almacenar los resultados
    res = {}
    #nos guardamos la longitud de data
    size = len(data)
    #creamos un diccionario para los resultados
    res_dict = {}
    
    for i in tqdm(range(0, size, batch_size)):
        #creamos la variable batch_text que contendra los textos de data
        batch_text = data[i:i+batch_size]
        #obtenemos la predicción del modelo
        results = model(batch_text, truncation=cfg['truncation'])
        #recorremos cada instancia de los resultados
        for j, result in enumerate(results):
            #obtenemos el indice de la clase con mayor score
            for cat in result:
                print(cat)
 


### Evaluación del modelo

En este caso no ha sido necesario entrenar el modelo, no obstante, lo evaluaremos en un conjunto reviews para los que se conoce su categoría de modo que podamos estimar el desempeño en nuevos datos.

Ejecute la siguiente casilla para evaluar el modelo en la porción de validación separada previamente.

**Notar que:**

- la salida del modelo es un diccionario con 'label' y 'score'. Debemos formatearla para poder comparar con los valores de referencia.

- para evitar problemas relacionados con el consumo de memoria, se realizará la predicción de instancias por lotes. Además, se utilizará TruncationStrategy.ONLY_FIRST para indicar al pipeline que trunque las secuencias con longitud mayor a la recomendada por el modelo.

In [186]:
# predecir y evaluar conjunto de validación con el modelo
data = val
true_labels = data[class_col]

m_pred = predict_model(model, cfg, data[text_col].to_list(), batch_size=128, pref='m')

# el nombre de los campos dependerá de pref al llamar a predic_model y las clases. Ver comentarios en la definición de la función
evaluate_model(true_labels, m_pred['labels_m'], m_pred['scores_m_positive'], 'positiva')

print(true_labels)
print(m_pred['labels_m'])
m_pred['scores_m_positive']


print('Done!')

 11%|█         | 1/9 [00:26<03:31, 26.50s/it]

{'label': 'NEGATIVE', 'score': 0.9916478395462036}
{'label': 'POSITIVE', 'score': 0.00835210457444191}
{'label': 'POSITIVE', 'score': 0.5562465786933899}
{'label': 'NEGATIVE', 'score': 0.4437534511089325}
{'label': 'POSITIVE', 'score': 0.9970722198486328}
{'label': 'NEGATIVE', 'score': 0.002927712630480528}
{'label': 'NEGATIVE', 'score': 0.8143700957298279}
{'label': 'POSITIVE', 'score': 0.1856299638748169}
{'label': 'POSITIVE', 'score': 0.9987713694572449}
{'label': 'NEGATIVE', 'score': 0.0012286314740777016}
{'label': 'POSITIVE', 'score': 0.718376874923706}
{'label': 'NEGATIVE', 'score': 0.28162312507629395}
{'label': 'NEGATIVE', 'score': 0.9692097306251526}
{'label': 'POSITIVE', 'score': 0.030790282413363457}
{'label': 'NEGATIVE', 'score': 0.9969608187675476}
{'label': 'POSITIVE', 'score': 0.003039152827113867}
{'label': 'NEGATIVE', 'score': 0.8170832395553589}
{'label': 'POSITIVE', 'score': 0.18291670083999634}
{'label': 'NEGATIVE', 'score': 0.9302888512611389}
{'label': 'POSITIVE'

 11%|█         | 1/9 [00:47<06:22, 47.77s/it]


KeyboardInterrupt: 

### Predicción de nuevos datos
Una vez evaluado el modelo para estimar su rendimiento en nuestro problema, podemos utilizarlo para predecir nuevos datos. En el ejemplo, utilizaremos la porción de prueba preparada inicialmente.

Ejecute la siguiente casilla para cargar los datos, descomentando las instrucciones necesarias según sea el caso.

In [11]:
# descomente las siguientes 3 líneas para leer datos desde Google Drive, asumiendo que se trata de un fichero llamado review.csv localizado dentro de una carpeta llamada 'Datos' en su Google Drive
#from google.colab import drive
#drive.mount('/content/drive')
#path = '/content/drive/MyDrive/Datos/ejemplo_review_train.csv'

# descomente la siguiente línea para leer los datos desde un archivo local, por ejemplo, asumiendo que se encuentra dentro de un directorio llamado sample_data
#path = './sample_data/ejemplo_review_train.csv'

# descomente la siguiente línea para leer datos desde un URL
path = 'https://github.com/TeachingTextMining/TextClassification/raw/main/02-SA-Transformers-Basic/sample_data/ejemplo_review_test.csv'

# leer los datos
new_data = pd.read_csv(path, sep=',')

print('Done!')

Done!


Ejecute la siguiente celda para clasificar los textos. Tenga en cuenta que, en dependencia del entorno de ejecución, la cantidad de textos y su longitud, la ejecución puede tardar varios minutos o requerir una cantidad de memoria no disponible.

In [12]:
# predecir los datos de prueba
m_pred = predict_model(model, cfg, new_data[text_col].to_list(), batch_size=128, pref='m')
pred_labels = m_pred['labels_m']

# obtener algunas estadísticas sobre la predicción en el conjunto de pruebas
categories = sorted(pred_labels.unique(), reverse=True)
hist = Counter(pred_labels.values) 

fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in categories]))
fig.show()

print('Done!')

100%|██████████| 2/2 [00:58<00:00, 29.05s/it]


Done!
